In [ ]:
import os
import cv2
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopandas import GeoSeries
from shapely.geometry import Polygon
from rasterio.windows import Window
from rasterio.plot import reshape_as_image
# import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

%matplotlib inline

In [24]:
PROJECT_DIR = '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data'

RASTER_PATH = os.path.join(PROJECT_DIR, 'T34JEP_20170101T082332/T34JEP_20170101T082332_TCI.jp2')
TRAIN_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.shp')
TEST_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/test/test.shp')
TRAIN_POLYGONS_CONVERTED = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.geojson')
RASTER_MASK_PATH = os.path.join(PROJECT_DIR, 'mask.jp2')
DROP_CSV_PATH = os.path.join(PROJECT_DIR, 'drop.csv')
FRAGMENT_STORAGE = os.path.join(PROJECT_DIR, 'split')
CROPPED_IMAGES = os.path.join(PROJECT_DIR, 'images_cropped_rgb')

In [25]:
# read dropped csv 
drop_df = pd.read_csv(DROP_CSV_PATH)
drop_df

,images_to_drop
0,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
1,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
2,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
3,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
4,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
...,...
1507,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
1508,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
1509,/home/mikolaj/github/quantum_geo/warsaw_ws/war...
1510,/home/mikolaj/github/quantum_geo/warsaw_ws/war...


In [26]:
# get num of dropped masks
drop_list = drop_df['images_to_drop'].tolist()

In [59]:
# masks and images paths
imgs_path = os.path.join(FRAGMENT_STORAGE, 'images')
masks_path = os.path.join(FRAGMENT_STORAGE, 'masks')

# only used masks 
mask_names = os.listdir(masks_path)
used_masks = [mask_name for mask_name in mask_names if mask_name not in drop_list]

# only used images
used_image_names = [os.path.join(imgs_path, mask_name.replace('mask_', "T34JEP_20170101T082332_TCI_")) for mask_name in used_masks]
used_masks = [os.path.join(masks_path, img_path) for img_path in used_masks]



Unet architecture

![title](images/unet.png)

In [60]:
def build_model(input_shape):
    'build unet model from scratch'
    inputs = Input(input_shape)

    conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    
    conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)

    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=-1)
    conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=-1)
    conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv9)    
    
    outputs = Conv2D(1, (1,1), activation = 'sigmoid')(conv9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [61]:
# add early stopping and model save
early_stopping = EarlyStopping(patience=5, verbose=1, monitor='dice_coef', mode='max')
model_checkpoint = ModelCheckpoint("model.hdf5", save_best_only=True, verbose=1, monitor='dice_coef', mode='max')

In [62]:
# read image and mask
def load_image(img_path):
      
    img = cv2.imread(img_path)

    return img

def load_mask(img_path):
      
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img[img>1]=1
        
    return img

In [63]:
#generate X
def generate_X(img_list):
    X = np.empty((len(img_list), 256, 256, 3))

    for i, item in enumerate(img_list):
        img = load_image(item)
        X[i,] = img

    return X

In [64]:
#generate Y
def generate_y(mask_list):
    y = np.empty((len(mask_list), 256, 256, 1), dtype=int)

    for i, item in enumerate(mask_list):
        mask = load_mask(item)
        y[i, :, : , 0] = mask
    
    return y.astype(np.float32)

In [65]:
# loss and metrics
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return 0.2 * binary_crossentropy(y_true, y_pred) + 0.8 * dice_loss(y_true, y_pred)

In [66]:
used_masks

['/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_36_27.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_19_8.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_0_36.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_0_13.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_34_17.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_22_13.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_26_0.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_4_32.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_1_39.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_14_33.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/masks/mask_2_11.png',
 '/home/mikolaj/github/quant

In [67]:
used_image_names

['/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_36_27.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_19_8.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_0_36.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_0_13.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_34_17.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_22_13.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_26_0.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_4_32.png',
 '/home/mikolaj/github/quantum_geo/warsaw_ws/warsaw_ws_data/split/images/T34JEP_20170101T082332_TCI_1_39.png'

In [68]:
# generate X
X = generate_X(used_image_names)

In [52]:
# generate Y
Y = generate_y(used_masks)

In [69]:
# build model with (256, 256, 3) input 
model = build_model((256, 256, 3))

In [70]:
# check model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 8)  224         ['input_2[0][0]']                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 256, 256, 8)  584         ['conv2d_19[0][0]']              
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 128, 128, 8)  0          ['conv2d_20[0][0]']        

In [71]:
# compile model
model.compile(optimizer=Adam(learning_rate = 1e-4),
              loss=bce_dice_loss,
              metrics=[dice_coef])

In [72]:
# run train
model.fit(X,
          Y,
          batch_size=8, 
          epochs=25,
          verbose=1,
          callbacks=[early_stopping,
                     model_checkpoint])

Epoch 1/25


2023-06-12 20:49:48.474262: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1387266048 exceeds 10% of free system memory.
2023-06-12 20:49:48.955317: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 462422016 exceeds 10% of free system memory.


221/221 [==============================] - ETA: 0s - loss: 2.7959 - dice_coef: 0.0141
Epoch 1: dice_coef improved from -inf to 0.01411, saving model to model.hdf5
221/221 [==============================] - 88s 389ms/step - loss: 2.7959 - dice_coef: 0.0141
Epoch 2/25
221/221 [==============================] - ETA: 0s - loss: 0.7847 - dice_coef: 0.0396
Epoch 2: dice_coef improved from 0.01411 to 0.03964, saving model to model.hdf5
221/221 [==============================] - 87s 392ms/step - loss: 0.7847 - dice_coef: 0.0396
Epoch 3/25
221/221 [==============================] - ETA: 0s - loss: 0.7719 - dice_coef: 0.0466
Epoch 3: dice_coef improved from 0.03964 to 0.04662, saving model to model.hdf5
221/221 [==============================] - 85s 385ms/step - loss: 0.7719 - dice_coef: 0.0466
Epoch 4/25
221/221 [==============================] - ETA: 0s - loss: 0.7338 - dice_coef: 0.0990
Epoch 4: dice_coef improved from 0.04662 to 0.09900, saving model to model.hdf5
221/221 [==================

In [ ]:
# load model from disk need to add custom objects bce_dice_loss and dice_coef
model = load_model('model.hdf5', custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef': dice_coef})

In [ ]:
# take some random image from training 

# predict 

# binarize prediction


# plot the results with matplotlib
f, axarr = plt.subplots(1, 3)
f.set_size_inches(15, 15)
axarr[0].imshow(Y[idx].reshape((256,256)))
axarr[1].imshow(test.reshape((256,256)))
axarr[2].imshow(X[idx].astype(np.uint8))